In [1]:
import tensorflow as tf
import os
import sys
import numpy as np

In [2]:
tf_path = '/home/average-joe/coding/libs/tensorflow/tensorflow/examples/speech_commands/'
sys.path.append(tf_path)

In [3]:
import input_data
import models

In [4]:
DATASET_DIR = '/path/to/dataset'

## Set model parameters

In [12]:
WANTED_WORDS = "your,custom,words"
TRAINING_STEPS = "12000,3000"
LEARNING_RATE = "0.001,0.0001"
MODEL_ARCHITECTURE = "tiny_conv"
TOTAL_STEPS = str(sum(map(int, TRAINING_STEPS.split(','))))

In [13]:
# Calculate the percentage of 'silence' and 'unknown' training samples required
# to ensure that we have equal number of samples for each label.
number_of_labels = WANTED_WORDS.count(',') + 1
number_of_total_labels = number_of_labels + 2 # for 'silence' and 'unknown' label
equal_percentage_of_training_samples = int(100.0/(number_of_total_labels))
SILENT_PERCENTAGE = equal_percentage_of_training_samples
UNKNOWN_PERCENTAGE = equal_percentage_of_training_samples

# Constants used during training only
VERBOSITY = "DEBUG"
EVAL_STEP_INTERVAL = '1000'
SAVE_STEP_INTERVAL = '1000'

# Constants for training directories and filepaths
LOGS_DIR = 'logs/'
TRAIN_DIR = 'train/' # for training checkpoints and other files.

# Constants for inference directories and filepaths
MODELS_DIR = 'models'
if not os.path.exists(MODELS_DIR):
    os.mkdir(MODELS_DIR)
MODEL_TF = os.path.join(MODELS_DIR, 'KWS_custom.pb')
MODEL_TFLITE = os.path.join(MODELS_DIR, 'KWS_custom.tflite')
FLOAT_MODEL_TFLITE = os.path.join(MODELS_DIR, 'KWS_custom_float.tflite')
MODEL_TFLITE_MICRO = os.path.join(MODELS_DIR, 'KWS_custom.cc')
SAVED_MODEL = os.path.join(MODELS_DIR, 'KWS_custom_saved_model')

In [14]:
# Constants which are shared during training and inference
PREPROCESS = 'micro'
WINDOW_STRIDE = 20

# Constants for Quantization
QUANT_INPUT_MIN = 0.0
QUANT_INPUT_MAX = 26.0
QUANT_INPUT_RANGE = QUANT_INPUT_MAX - QUANT_INPUT_MIN

# Constants for audio process during Quantization and Evaluation
SAMPLE_RATE = 16000
CLIP_DURATION_MS = 1000
WINDOW_SIZE_MS = 30.0
FEATURE_BIN_COUNT = 40
BACKGROUND_FREQUENCY = 0.8
BACKGROUND_VOLUME_RANGE = 0.1
TIME_SHIFT_MS = 100.0

# Use the custom local dataset and set the tes/val/train split
DATA_URL = ''
VALIDATION_PERCENTAGE = 10
TESTING_PERCENTAGE = 10

## Run commands in terminal to use conda envs

In [15]:
train_command = f'python {os.path.join(tf_path, "train.py")} \
--data_dir={DATASET_DIR} \
--data_url={DATA_URL} \
--wanted_words={WANTED_WORDS} \
--silence_percentage={SILENT_PERCENTAGE} \
--unknown_percentage={UNKNOWN_PERCENTAGE} \
--preprocess={PREPROCESS} \
--window_stride={WINDOW_STRIDE} \
--model_architecture={MODEL_ARCHITECTURE} \
--how_many_training_steps="{TRAINING_STEPS}" \
--learning_rate="{LEARNING_RATE}" \
--train_dir={TRAIN_DIR} \
--summaries_dir={LOGS_DIR} \
--verbosity={VERBOSITY} \
--eval_step_interval={EVAL_STEP_INTERVAL} \
--save_step_interval={SAVE_STEP_INTERVAL}'
print(train_command)

python /home/average-joe/coding/libs/tensorflow/tensorflow/examples/speech_commands/train.py --data_dir=/path/to/dataset --data_url= --wanted_words=your,custom,words --silence_percentage=20 --unknown_percentage=20 --preprocess=micro --window_stride=20 --model_architecture=tiny_conv --how_many_training_steps="12000,3000" --learning_rate="0.001,0.0001" --train_dir=train/ --summaries_dir=logs/ --verbosity=DEBUG --eval_step_interval=1000 --save_step_interval=1000


In [9]:
#!python {train_command}

In [10]:
save_command = f'python {os.path.join(tf_path, "freeze.py")} \
--wanted_words={WANTED_WORDS} \
--window_stride_ms={WINDOW_STRIDE} \
--preprocess={PREPROCESS} \
--model_architecture={MODEL_ARCHITECTURE} \
--start_checkpoint={os.path.join(TRAIN_DIR,MODEL_ARCHITECTURE)}.ckpt-{TOTAL_STEPS} \
--save_format=saved_model \
--output_file={SAVED_MODEL}'
print(save_command)

python /home/average-joe/coding/libs/tensorflow/tensorflow/examples/speech_commands/freeze.py --wanted_words=on,off,left --window_stride_ms=20 --preprocess=micro --model_architecture=tiny_conv --start_checkpoint=train/tiny_conv.ckpt-15000 --save_format=saved_model --output_file=models/KWS_custom_saved_model


In [11]:
#!python {save_command}

## Build TFLite model

In [27]:
model_settings = models.prepare_model_settings(
    len(input_data.prepare_words_list(WANTED_WORDS.split(','))),
    SAMPLE_RATE,
    CLIP_DURATION_MS,
    WINDOW_SIZE_MS,
    WINDOW_STRIDE,
    FEATURE_BIN_COUNT,
    PREPROCESS
)

In [30]:
audio_processor = input_data.AudioProcessor(
    DATA_URL,
    DATASET_DIR,
    SILENT_PERCENTAGE,
    UNKNOWN_PERCENTAGE,
    WANTED_WORDS.split(','),
    VALIDATION_PERCENTAGE,
    TESTING_PERCENTAGE,
    model_settings,
    LOGS_DIR
)

2022-07-24 14:22:35.795979: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-24 14:22:35.799882: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-24 14:22:35.800267: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-24 14:22:35.800902: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [42]:
REP_DATA_SIZE = 58
#with tf.Session() as sess:
with tf.compat.v1.Session() as sess: #replaces the above line for use with TF2.x
    float_converter = tf.lite.TFLiteConverter.from_saved_model(SAVED_MODEL)
    float_tflite_model = float_converter.convert()
    float_tflite_model_size = open(FLOAT_MODEL_TFLITE, "wb").write(float_tflite_model)
    print("Float model is %d bytes" % float_tflite_model_size)

    converter = tf.lite.TFLiteConverter.from_saved_model(SAVED_MODEL)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    #converter.inference_input_type = tf.lite.constants.INT8
    converter.inference_input_type = tf.compat.v1.lite.constants.INT8 #replaces the above line for use with TF2.x   
    #converter.inference_output_type = tf.lite.constants.INT8
    converter.inference_output_type = tf.compat.v1.lite.constants.INT8 #replaces the above line for use with TF2.x
    def representative_dataset_gen():
        for i in range(REP_DATA_SIZE):
            data, _ = audio_processor.get_data(1, i*1, model_settings,
                                             BACKGROUND_FREQUENCY, 
                                             BACKGROUND_VOLUME_RANGE,
                                             TIME_SHIFT_MS,
                                             'testing',
                                             sess)
            flattened_data = np.array(data.flatten(), dtype=np.float32).reshape(1, 1960)
            print(i)
            yield [flattened_data]
    converter.representative_dataset = representative_dataset_gen
    tflite_model = converter.convert()
    tflite_model_size = open(MODEL_TFLITE, "wb").write(tflite_model)
    print("Quantized model is %d bytes" % tflite_model_size)


2022-07-24 14:38:41.340008: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-24 14:38:41.340406: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-24 14:38:41.340805: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-24 14:38:41.341160: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-24 14:38:41.341473: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

INFO:tensorflow:Restoring parameters from models/KWS_custom_saved_model/variables/variables


2022-07-24 14:38:41.343002: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-24 14:38:41.343339: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-24 14:38:41.343643: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-24 14:38:41.343979: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-24 14:38:41.344283: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

INFO:tensorflow:The given SavedModel MetaGraphDef contains SignatureDefs with the following keys: {'serving_default'}


INFO:tensorflow:The given SavedModel MetaGraphDef contains SignatureDefs with the following keys: {'serving_default'}


...

INFO:tensorflow:Restoring parameters from models/KWS_custom_saved_model/variables/variables
2022-07-24 14:38:41.676151: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-24 14:38:41.676413: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2022-07-24 14:38:41.676473: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2022-07-24 14:38:41.676700: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-24 14:38:41.676940: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

0


2022-07-24 14:38:41.694895: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-24 14:38:41.695190: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2022-07-24 14:38:41.695249: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2022-07-24 14:38:41.695453: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-24 14:38:41.695744: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-24 14:38:41.695977: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] su

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
Quantized model is 22920 bytes


fully_quantize: 0, inference_type: 6, input_inference_type: 9, output_inference_type: 9
2022-07-24 14:38:41.921472: I tensorflow/compiler/mlir/lite/flatbuffer_export.cc:1972] Estimated count of arithmetic ops: 0.684 M  ops, equivalently 0.342 M  MACs



In [55]:
def run_tflite_inference_testSet(tflite_model_path, model_type="Float"):
    np.random.seed(0)
    with tf.compat.v1.Session() as sess:
    # with tf.compat.v1.Session() as sess: #replaces the above line for use with TF2.x
        test_data, test_labels = audio_processor.get_data(
            -1, 0, model_settings, BACKGROUND_FREQUENCY, BACKGROUND_VOLUME_RANGE,
            TIME_SHIFT_MS, 'testing', sess)
    test_data = np.expand_dims(test_data, axis=1).astype(np.float32)

    #
    # Initialize the interpreter
    #
    interpreter = tf.lite.Interpreter(tflite_model_path)
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]

    #
    # For quantized models, manually quantize the input data from float to integer
    #
    if model_type == "Quantized":
        input_scale, input_zero_point = input_details["quantization"]
        test_data = test_data / input_scale + input_zero_point
        test_data = test_data.astype(input_details["dtype"])

    #
    # Evaluate the predictions
    #
    correct_predictions = 0
    for i in range(len(test_data)):
        interpreter.set_tensor(input_details["index"], test_data[i])
        interpreter.invoke()
        output = interpreter.get_tensor(output_details["index"])[0]
        top_prediction = output.argmax()
        correct_predictions += (top_prediction == test_labels[i])

    print('%s model accuracy is %f%% (Number of test samples=%d)' % (
    model_type, (correct_predictions * 100) / len(test_data), len(test_data)))

In [56]:
#run_tflite_inference_testSet(FLOAT_MODEL_TFLITE)
#
#run_tflite_inference_testSet(MODEL_TFLITE, model_type="Quantized")

2022-07-24 14:51:36.424863: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-24 14:51:36.425530: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-24 14:51:36.426021: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-24 14:51:36.426643: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-24 14:51:36.427143: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

Float model accuracy is 94.915254% (Number of test samples=59)
Quantized model accuracy is 94.915254% (Number of test samples=59)


In [57]:
!xxd -i {MODEL_TFLITE} > {MODEL_TFLITE_MICRO}
REPLACE_TEXT = MODEL_TFLITE.replace('/', '_').replace('.', '_')
!sed -i 's/'{REPLACE_TEXT}'/g_model/g' {MODEL_TFLITE_MICRO}

In [58]:
!cat {MODEL_TFLITE_MICRO}

unsigned char g_model[] = {
  0x20, 0x00, 0x00, 0x00, 0x54, 0x46, 0x4c, 0x33, 0x00, 0x00, 0x00, 0x00,
  0x14, 0x00, 0x20, 0x00, 0x1c, 0x00, 0x18, 0x00, 0x14, 0x00, 0x10, 0x00,
  0x0c, 0x00, 0x00, 0x00, 0x08, 0x00, 0x04, 0x00, 0x14, 0x00, 0x00, 0x00,
  0x1c, 0x00, 0x00, 0x00, 0x1c, 0x00, 0x00, 0x00, 0x74, 0x00, 0x00, 0x00,
  0x70, 0x52, 0x00, 0x00, 0x80, 0x52, 0x00, 0x00, 0xe8, 0x58, 0x00, 0x00,
  0x03, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x02, 0x00, 0x00, 0x00,
  0x34, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0xdc, 0xff, 0xff, 0xff,
  0x0e, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0x13, 0x00, 0x00, 0x00,
  0x43, 0x4f, 0x4e, 0x56, 0x45, 0x52, 0x53, 0x49, 0x4f, 0x4e, 0x5f, 0x4d,
  0x45, 0x54, 0x41, 0x44, 0x41, 0x54, 0x41, 0x00, 0x08, 0x00, 0x0c, 0x00,
  0x08, 0x00, 0x04, 0x00, 0x08, 0x00, 0x00, 0x00, 0x0d, 0x00, 0x00, 0x00,
  0x04, 0x00, 0x00, 0x00, 0x13, 0x00, 0x00, 0x00, 0x6d, 0x69, 0x6e, 0x5f,
  0x72, 0x75, 0x6e, 0x74, 0x69, 0x6d, 0x65, 0x5f, 0x76, 0x65, 0x72, 0x73,
  0x69, 0x